In [ ]:
import gym 
import random 

In [ ]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n


In [ ]:
episodes = 10 
for episode in range(1, episodes):
  state = env.reset()
  done = False 
  score = 0 
  while not done:
    action = random.choice([0, 1])
    n_state, reward, done, info = env.step(action) 
    score += reward
  print(f'episode: {episode}, score: {score} ')

episode: 1, score: 18.0 
episode: 2, score: 28.0 
episode: 3, score: 12.0 
episode: 4, score: 11.0 
episode: 5, score: 10.0 
episode: 6, score: 56.0 
episode: 7, score: 12.0 
episode: 8, score: 20.0 
episode: 9, score: 15.0 


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(states, actions):
  model = Sequential()
  model.add(Flatten(input_shape=(1, states)))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(actions, activation='linear'))
  return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 128)               640       
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                                 
Total params: 17,410
Trainable params: 17,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!pip install keras-rl2

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory 

In [ ]:
def build_agent(model, actions):
  policy = BoltzmannGumbelQPolicy()
  memory = SequentialMemory(limit=50000, window_length=1)
  dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                 nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
  return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 4:42:21 - reward: 1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 139s 14ms/step - reward: 1.0000
88 episodes - episode_reward: 113.511 [8.000, 500.000] - loss: 0.535 - mae: 18.752 - mean_q: 37.699

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 134s 13ms/step - reward: 1.0000
44 episodes - episode_reward: 224.591 [156.000, 323.000] - loss: 0.470 - mae: 35.769 - mean_q: 71.789

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 134s 13ms/step - reward: 1.0000
30 episodes - episode_reward: 332.233 [111.000, 500.000] - loss: 1.035 - mae: 38.638 - mean_q: 77.504

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 131s 13ms/step - reward: 1.0000
25 episodes - episode_reward: 401.240 [218.000, 500.000] - loss: 3.578 - mae: 42.987 - mean_q: 86.162

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 139s 14ms/step - reward: 1.0000
done, took 676.603 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)

Testing for 100 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
Episode 21: reward: 500.000, steps: 500
Episode 22: reward: 500.000, steps: 500
Episode 23: reward: 500.000, steps: 500
Episode 24: reward: 500.000, steps: 500
Episode 25: reward: 

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)